In [ ]:
!pip install openai
!pip install langchain
!pip install langchain_community
!pip install langchain_opneai

In [ ]:
import os
import openai

#이전의 방식은 소스코드를 내가 업로드 했을 떄, 다른 사람들이 내 키를 볼 수 있음
#'이름'==userdata.get(이름),값=내 open_api_key
from google.colab import userdata
os.environ['OPENAI_API_KEY']=userdata.get('OPEN_AI_KEY')

openai.api_key=os.getenv('OPENAI_API_KEY')

In [ ]:
#대화형 프롬프트의 양식을 지정
from langchain.prompts.chat import ChatPromptTemplate

#모델의 결과값을 원하는 방식으로 파싱(형태 변형)
from langchain.schema import BaseOutputParser

In [ ]:
chat_prompt_template=ChatPromptTemplate.from_messages([
    #openai처럼 role-content의 순서로 튜플을 전달하여 요청
    ("system","댓글에 대해 감정 분석을 수행할 것,주어진 user_input에 대해 긍정적이면 1 부정적이면 0,오로지 0과 1로만 답할 것 "),
    ("human","{input_text}"),
])

In [ ]:
#BaseoutputParser를 상속하여 새로운 클랫인 ClassificationOutputParser를 정의
#이 클래스는 결과에 대해, str형태의 1과 0을 각각 정수 1과 0으로 변환하는 역할을 함

class ClassificationOutputParser(BaseOutputParser):
  def parse(self, text:str):
    if '1' in text:
      return 1
    return 0

In [ ]:
#LangChain에서 OpenAI의 대화형 모델을 사용하여 대화를 생성
from langchain.chat_models import ChatOpenAI

#langchain에서 openai랑 대화할 모델의 정의
chat_model=ChatOpenAI()

#chain을 통해서 양식 생성 -> 모델에 전달 -> 결과값 정리 까지를 한꺼번에 처리
#처리하는 함수를 객체로 사용함
#chain이라는 함수로 다시 생성
chain=chat_prompt_template | chat_model  |ClassificationOutputParser()

In [ ]:
def langchain_llm(input_text):
    output=chain.invoke({'input_text':input_text})
    return output
    #output 0과 1의 값을 가짐

In [ ]:
def classify_text(input_text):
    output=langchain_llm(input_text)
    return output
    #output=input_text 데이터를 lln 모델에 돌린값
    #두 겹으로 함수를

###분석에 사용할 데이터 로드

In [ ]:
import urllib.request


urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/tykimos/tykimos.github.io/master/warehouse/dataset/tarr_train.txt",
    filename="tarr_train.txt",
)

In [ ]:
import pandas as pd

#delimiter 어떤 걸로 분류되어 있는가?
df=pd.read_csv("tarr_train.txt",delimiter='\t')

In [ ]:
df

In [ ]:
actual_labels=[]
predicted_labels=[]

total=len(df)

for index,row in df.iterrows():
    comment=row['comment']
    actual_label=row['label']

    #llm의 질의 응답을 받아옴
    predicted_label=classify_text(comment)

    actual_labels.append(actual_label)
    predicted_labels.append(predicted_label)

    print(f"[{index+1}/{total}]") #전체에서 몇 번째 실행?
    print('comment',comment)  #댓글 내용
    print('actual',actual_label) #실제 라벨
    print('predict',predicted_label) #llm이 나눈 라벨

    if index>10:
        break

## 댓글에 대한 답변 생성

In [ ]:
chat_prompt_reply=ChatPromptTemplate.from_messages([
    ('system',"너는 친절한 20대 사장님이야. 긍정적인 댓글에는 감사함을 구체적으로 표현하고 부정적인 댓글에는 미안함을 표현하면서 개선사항을 함께 말해줘.대신 충청도 사투리를 사용해줘"),
    ('human','{input_text}')
])


In [ ]:
from langchain.schema.output_parser import StrOutputParser

#답변 맥락 정의->모델에 넣기->결과만 나오게끔
chain=chat_prompt_reply | chat_model | StrOutputParser()

#인풋 데이터에 대한 답변을 생성하는 함수 생성
def generate_reply(input_text):
    output=langchain_llm(input_text)
    return output

In [ ]:
for index,row in df.iterrows():
    comment=row['comment']

    #함수를 이용해서 댓글에 대한 답변 생성
    reply=generate_reply(comment)

    print(f"[{index+1}/{total}]")
    print('comment',comment)
    print("reply : ", reply)
    print("---------------")

    if index > 3:
        break